# test function list

In this part, we will show gradient descent optimization on various 2-dimensioanl test function from this wiki page [wiki/test-function-for-optimization](https://en.wikipedia.org/wiki/Test_functions_for_optimization).

We will plot the function landscape around the optimal point and also the optimization path.

We choose the initial point for optimization uniformly from the range $[-1, 1]$, so it could be biased to the local optimal point around the origin.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

import numqi

np_rng = np.random.default_rng()
hf_uniform_para = lambda *x: torch.nn.Parameter(torch.tensor(np_rng.uniform(-1, 1, size=x), dtype=torch.float64))

In [ ]:
def hf_demo(model, num_repeat=3, xlim=None, ylim=None, x_optim=None, tag_logscale=False):
    theta_optim,info = numqi.optimize.minimize(model, tol=1e-10, num_repeat=num_repeat, tag_record_path=True)
    path = np.stack(info['path'])
    print(f'optimal theta: {theta_optim.x}')
    print(f'optimal loss: {theta_optim.fun}')

    hf_model = numqi.optimize.hf_model_wrapper(model)
    xdata = np.linspace(*xlim, 100)
    ydata = np.linspace(*ylim, 100)
    if len(x_optim)==2:
        zdata = np.array([[hf_model(np.array([x, y]), tag_grad=False) for x in xdata] for y in ydata])
    else:
        zdata = np.array([[hf_model(np.concatenate([np.array([x, y]),x_optim[2:]]), tag_grad=False) for x in xdata] for y in ydata])
    if tag_logscale:
        zdata = np.log(np.maximum(1e-4,zdata))
    fig,ax = plt.subplots()
    contour_set = ax.contourf(xdata, ydata, zdata, levels=15, cmap='winter')
    fig.colorbar(contour_set)
    ax.plot([x_optim[0]], [x_optim[1]], 'rx', label='optimal')
    ax.plot(path[:,0], path[:,1], '-', color='orange', label='path')
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.legend()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    if tag_logscale:
        ax.set_title('log scale plot of loss function')
    fig.tight_layout()

## Rastrigin function

$$ f(x)=An+\sum_{i=1}^n [x_i^2-A\cos(2\pi x_i)], A=10$$

$$ f(0,\cdots,0)=0 $$

In [ ]:
class Rastrigin(torch.nn.Module):
    def __init__(self, n):
        super().__init__()
        self.x = hf_uniform_para(n)
        self.A = 10
        # solution [0,0,...,0], 0

    def forward(self):
        x = self.x
        loss = self.A*x.shape[0] + (x*x - self.A*torch.cos(2*np.pi*x)).sum()
        return loss
    
n = 2
model = Rastrigin(n)
x_optim = np.zeros(n)
hf_demo(model, num_repeat=10, xlim=(-5.12, 5.12), ylim=(-5.12, 5.12), x_optim=x_optim)
# when n is large, it's almost impossible to find the global minimum

## Ackley function

$$ f(x,y)=-20\mathrm{exp}[-0.2\sqrt{0.5(x^2+y^2)}] - \mathrm{exp}[0.5(\cos(2\pi x)+\cos(2\pi y))] + e + 20 $$

$$ f(0,0)=0 $$

In [ ]:
class Ackley(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.theta = hf_uniform_para(2)
        # solution [0,0] 0

    def forward(self):
        x,y = self.theta
        tmp0 = -20*torch.exp(-0.2*torch.sqrt(0.5*(x*x + y*y)))
        tmp1 = -torch.exp(0.5*(torch.cos(2*np.pi*x)+torch.cos(2*np.pi*y))) + np.e + 20
        ret = tmp0 + tmp1
        return ret

model = Ackley()
x_optim = np.zeros(2)
hf_demo(model, num_repeat=10, xlim=(-5, 5), ylim=(-5, 5), x_optim=x_optim)

## Rosenbrock function

$$ f(x)=\sum_{i=1}^{n-1}[100(x_{i+1}-x_i^2)^2 + (1-x_i)^2] $$

$$ f(1,\cdots,1)=0 $$

In [ ]:
class Rosenbrock(torch.nn.Module):
    def __init__(self, n):
        super().__init__()
        self.theta = hf_uniform_para(n)
        # solution [1,1,...,1] 0

    def forward(self):
        tmp0 = self.theta[1:] - self.theta[:-1]**2
        tmp1 = 1-self.theta[:-1]
        ret = 100*torch.dot(tmp0, tmp0) + torch.dot(tmp1,tmp1)
        return ret
    
n = 2
model = Rosenbrock(n)
x_optim = np.ones(n)
hf_demo(model, num_repeat=10, xlim=(-2, 2), ylim=(-1, 3), x_optim=x_optim, tag_logscale=True)

## Beale function

$$ f(x,y)=(1.5-x+xy)^2 + (2.25-x+xy^2)^2 + (2.625-x+xy^3)^2 $$

$$ f(3,0.5)=0 $$

In [ ]:
class Beale(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [3,0.5] 0

    def forward(self):
        x,y = self.theta
        ret = (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2 + (2.625 - x + x*y**3)**2
        return ret

model = Beale()
x_optim = np.array([3, 0.5])
hf_demo(model, num_repeat=10, xlim=(-4.5, 4.5), ylim=(-4.5, 4.5), x_optim=x_optim, tag_logscale=True)

## Goldstein-Price function

$$ f(x,y)=[1+(x+y+1)^2(19-14x+3x^2-14y+6xy+3y^2)][30+(2x-3y)^2(18-32x+12x^2+48y-36xy+27y^2)]-3 $$

$$ f(0,-1)=0 $$

comment: the function value is shifted by a constant $3$ for good plotting.

In [ ]:
class GoldsteinPrice(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [0,-1] 0

    def forward(self):
        x,y = self.theta
        tmp0 = 1 + (x+y+1)**2 * (19-14*x+3*x*x-14*y+6*x*y+3*y*y)
        tmp1 = 30 + (2*x-3*y)**2 * (18-32*x+12*x*x+48*y-36*x*y+27*y*y)
        ret = tmp0*tmp1 - 3 #shift 3 for good plotting
        return ret

model = GoldsteinPrice()
x_optim = np.array([0, -1])
hf_demo(model, num_repeat=10, xlim=(-2, 2), ylim=(-3, 1), x_optim=x_optim, tag_logscale=True)

## Booth function

$$ f(x,y)=(x+2y-7)^2 + (2x+y-5)^2 $$

$$ f(1,3)=0 $$

In [ ]:
class Booth(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [1,3] 0

    def forward(self):
        x,y = self.theta
        tmp0 = x+2*y-7
        tmp1 = 2*x+y-5
        ret = tmp0*tmp0 + tmp1*tmp1
        return ret

model = Booth()
x_optim = np.array([1, 3])
hf_demo(model, num_repeat=10, xlim=(-10, 10), ylim=(-10, 10), x_optim=x_optim, tag_logscale=True)

## Bukin Function N.6

$$ f(x,y)=100\sqrt{|y-0.01x^2|}+0.01|x+10| $$

$$ f(-10,1)=0 $$

In [ ]:
class BukinFunctionN6(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [-10,1] 0

    def forward(self):
        x,y = self.theta
        # possible numerical unstable here, also hard for Adam
        ret = 100*torch.sqrt(torch.abs(y-0.01*x*x)) + 0.01*torch.abs(x+10)
        return ret

model = BukinFunctionN6()
x_optim = np.array([-10, 1])
hf_demo(model, num_repeat=10, xlim=(-15, -5), ylim=(-4, 6), x_optim=x_optim, tag_logscale=True)

## Matyas function

$$ f(x,y)=0.26(x^2+y^2)-0.48xy $$

$$ f(0,0)=0 $$

In [ ]:
class Matyas(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [-0,0] 0

    def forward(self):
        x,y = self.theta
        ret = 0.26*(x*x + y*y) - 0.48*x*y
        return ret

model = Matyas()
x_optim = np.array([0, 0])
hf_demo(model, num_repeat=10, xlim=(-10, 10), ylim=(-10, 10), x_optim=x_optim, tag_logscale=True)

## Levi function N.13

$$ f(x,y)=\sin^2(3\pi x)+ (x-1)^2(1+\sin^2(3\pi y)) + (y-1)^2(1+\sin^2(2\pi y)) $$

$$ f(1,1)=0 $$

In [ ]:
class LeviFunction(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [0,0] 0

    def forward(self):
        x,y = self.theta
        ret = torch.sin(3*np.pi*x)**2 + (x-1)**2*(1+torch.sin(3*np.pi*y)**2) + (y-1)**2*(1+torch.sin(2*np.pi*y)**2)
        return ret

model = LeviFunction()
x_optim = np.array([0, 0])
hf_demo(model, num_repeat=10, xlim=(-10, 10), ylim=(-10, 10), x_optim=x_optim)

## Himmelblau's function

$$ f(x,y)=(x^2+y-11)^2 + (x+y^2-7)^2 $$

$$ f(3,2)=f(-2.805118,3.131312)=f(-3.779310,-3.283186)=f(3.584428,-1.848126)=0 $$

In [ ]:
class Himmelblau(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [3,2],[-2.805118,3.131312],[-3.779310,-3.283186],[3.584428,-1.848126] 0

    def forward(self):
        x,y = self.theta
        tmp0 = x*x + y - 11
        tmp1 = x + y*y - 7
        ret = tmp0*tmp0 + tmp1*tmp1
        return ret

model = Himmelblau()
x_optim = np.array([[3, 2],[-2.805118,3.131312],[-3.779310,-3.283186],[3.584428,-1.848126]])
hf_demo(model, num_repeat=10, xlim=(-5, 5), ylim=(-5, 5), x_optim=x_optim[0], tag_logscale=True)

## Three-hump camel function

$$ f(x,y)=2x^2-1.05x^4+\frac{x^6}{6}+xy+y^2 $$

$$ f(0,0)=0 $$

In [ ]:
class ThreeHumpCamel(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [0,0] 0

    def forward(self):
        x,y = self.theta
        ret = 2*x*x - 1.05*x*x*x*x + x*x*x*x*x*x/6 + x*y + y*y
        return ret

model = ThreeHumpCamel()
x_optim = np.array([0, 0])
hf_demo(model, num_repeat=10, xlim=(-5, 5), ylim=(-5, 5), x_optim=x_optim, tag_logscale=True)

## Easom function

$$ f(x,y)=-\cos(x)\cos(y)\exp[-(x-\pi)^2-(y-\pi)^2] $$

$$ f(\pi,\pi)=-1 $$

In [ ]:
class Easom(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [pi,pi] -1

    def forward(self):
        x,y = self.theta
        ret = -torch.cos(x)*torch.cos(y)*torch.exp(-(x-np.pi)**2-(y-np.pi)**2)
        return ret

model = Easom()
x_optim = np.array([np.pi, np.pi])
hf_demo(model, num_repeat=10, xlim=(-100, 100), ylim=(-100, 100), x_optim=x_optim)

## Cross-in-tray function

$$ f(x,y)=-0.0001\left[\left|\sin(x)\sin(y)\exp\left(\left|100-\frac{\sqrt{x^2+y^2}}{\pi}\right|\right)\right|+1\right]^{0.1} $$

$$ f(1.34941,-1.34941)=f(1.34941,1.34941)=f(-1.34941,1.34941)=f(-1.34941,-1.34941)=-2.06261 $$

In [ ]:
class CrossInTray(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [1.34941,-1.34941],[1.34941,1.34941],[-1.34941,1.34941],[-1.34941,-1.34941] -2.06261

    def forward(self):
        x,y = self.theta
        tmp0 = torch.abs(100 - torch.sqrt(x*x + y*y)/np.pi)
        tmp1 = torch.abs(torch.sin(x)*torch.sin(y)*torch.exp(tmp0))
        ret = -0.0001*tmp1**0.1
        return ret

model = CrossInTray()
x_optim = np.array([[1.34941,-1.34941],[1.34941,1.34941],[-1.34941,1.34941],[-1.34941,-1.34941]])
hf_demo(model, num_repeat=10, xlim=(-10, 10), ylim=(-10, 10), x_optim=x_optim[0])

## Eggholder function

$$ f(x,y)=-(y+47)\sin\sqrt{|x/2+(y+47)|}-x\sin\sqrt{|x-(y+47)|} $$

$$ f(512,404.2319)=-959.6407 $$

In [ ]:
class Eggholder(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [512,404.2319] -959.6407

    def forward(self):
        x,y = self.theta
        tmp0 = -(y+47)*torch.sin(torch.sqrt(torch.abs(x/2 + (y+47))))
        tmp1 = -x*torch.sin(torch.sqrt(torch.abs(x-(y+47))))
        ret = tmp0 + tmp1
        return ret

model = Eggholder()
x_optim = np.array([512, 404.2319])
hf_demo(model, num_repeat=10, xlim=(-1000, 1000), ylim=(-1000, 1000), x_optim=x_optim)

## Holder table function

$$ f(x,y)=-\left|\sin(x)\cos(y)\exp\left(\left|1-\frac{\sqrt{x^2+y^2}}{\pi}\right|\right)\right| $$

$$ f(8.05502,9.66459)=f(-8.05502, 9.66459)=f(8.05502, -9.66459)=f(-8.05502, -9.66459)=-19.2085 $$

In [ ]:
class HolderTable(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [8.05502,9.66459] -19.2085

    def forward(self):
        x,y = self.theta
        tmp0 = torch.sin(x)*torch.cos(y)
        tmp1 = torch.exp(torch.abs(1 - torch.sqrt(x*x + y*y)/np.pi))
        ret = -torch.abs(tmp0*tmp1)
        return ret

model = HolderTable()
x_optim = np.array([[8.05502, 9.66459], [-8.05502, 9.66459], [8.05502, -9.66459], [-8.05502, -9.66459]])
hf_demo(model, num_repeat=10, xlim=(-10, 10), ylim=(-10, 10), x_optim=x_optim[1])

## McCormick function

$$ f(x,y)=\sin(x+y)+(x-y)^2-1.5x+2.5y+1 $$

$$ f(-0.54719,-1.54719)=-1.9133 $$

In [ ]:
class McCormick(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [-0.54719,-1.54719] -1.9133

    def forward(self):
        x,y = self.theta
        tmp0 = torch.sin(x+y)
        tmp1 = (x-y)**2
        tmp2 = -1.5*x + 2.5*y + 1
        ret = tmp0 + tmp1 + tmp2
        return ret

# out of domain
model = McCormick()
x_optim = np.array([-0.54719,-1.54719])
hf_demo(model, num_repeat=10, xlim=(-2.5, 4), ylim=(-3, 4), x_optim=x_optim)

## Schaffer function N.2

$$ f(x,y)=0.5+\frac{\sin^2(x^2-y^2)-0.5}{[1+0.001(x^2+y^2)]^2} $$

$$ f(0,0)=0 $$

In [ ]:
class SchafferFunctionN2(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [0,0] 0

    def forward(self):
        x,y = self.theta
        tmp0 = torch.sin(x*x-y*y)
        tmp1 = 1 + 0.001*(x*x + y*y)
        ret = 0.5 + (tmp0*tmp0 - 0.5)/(tmp1*tmp1)
        return ret

model = SchafferFunctionN2()
x_optim = np.array([0, 0])
hf_demo(model, num_repeat=10, xlim=(-50, 50), ylim=(-50, 50), x_optim=x_optim)

## Schaffer function N.4

$$ f(x,y)=0.5+\frac{\cos^2(\sin|x^2-y^2|)-0.5}{[1+0.001(x^2+y^2)]^2} $$

$$ f(0,1.25313)=f(0,-1.25313)=f(1.25313,0)=f(-1.25313,0)=0.292579 $$

In [ ]:
class SchafferFunctionN4(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.theta = hf_uniform_para(2)
        # [0,1.25313] 0.292579

    def forward(self):
        x,y = self.theta
        tmp0 = torch.cos(torch.sin(torch.abs(x*x-y*y)))
        tmp1 = 1 + 0.001*(x*x + y*y)
        ret = 0.5 + (tmp0*tmp0 - 0.5)/(tmp1*tmp1)
        return ret

model = SchafferFunctionN4()
x_optim = np.array([[0, 1.25313], [0, -1.25313], [1.25313,0], [-1.25313,0]])
hf_demo(model, num_repeat=10, xlim=(-50, 50), ylim=(-50, 50), x_optim=x_optim[0])

## Styblinski-Tang function

$$ f(x)=\frac{1}{2}\sum_{i=1}^n[x_i^4-16x_i^2+5x_i] $$

$$ f(-2.903534,-2.903534,\cdots,-2.903534)=-39.16599n $$

In [ ]:
class StyblinskiTangFunction(torch.nn.Module):
    def __init__(self, n) -> None:
        super().__init__()
        self.theta = hf_uniform_para(n)
        # [-2.903534]*n -39.16599*n

    def forward(self):
        x = self.theta
        ret = 0.5*(x*x*x*x - 16*x*x + 5*x).sum()
        return ret

n = 2
model = StyblinskiTangFunction(n)
x_optim = np.array([-2.903534]*n)
hf_demo(model, num_repeat=10, xlim=(-5, 5), ylim=(-5, 5), x_optim=x_optim)